In [1]:
import os
import glob

import DESutils
import GPRutils
import vK2KGPR
import plotGPR
import vK2KGPR
import vonkarmanFT as vk

import numpy as np
import astropy.units as u
import astropy.table as tb
from astropy.time import Time
import matplotlib.pyplot as plt
plt.style.use('~/GitHub/custom-matplotlib/custom.mplstyle')
import astropy.stats as stats
from scipy.signal import savgol_filter
import scipy.linalg
import astropy.coordinates as co

from IPython import embed
from importlib import reload
import warnings
warnings.filterwarnings("ignore")

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
def radec_to_gnomonic(ra, dec, ra_0, dec_0):
	'''
	Converts RA, DEC to x,y in gnomonic coordinates using pixmappy so we can use an Euclidean metric for the FoF
	'''
	ra[np.where(ra > 180)] = ra[np.where(ra > 180)] - 360
	c_dec_0 = np.cos(np.pi*dec_0/180.)
	s_dec_0 = np.sin(np.pi*dec_0/180.)
	c_dec   = np.cos(np.pi*dec/180.)
	s_dec   = np.sin(np.pi*dec/180.)
	c_ra    = np.cos(np.pi*(ra-ra_0)/180.)
	s_ra    = np.sin(np.pi*(ra-ra_0)/180.)
	cos_c = s_dec_0*s_dec + c_dec_0*c_dec*c_ra
	x = c_dec*s_ra/cos_c
	y = (c_dec_0*s_dec - s_dec_0*c_dec*c_ra)/cos_c
	return 180*x/np.pi, 180*y/np.pi

In [3]:
def getXiStuff(dC, rMax):
    xi1, xi2 = dC.JackKnifeXi(rMax=rMax)
    xi0_raw, xi0_GPR = np.abs(xi1[0].value), np.abs(xi2[0].value)
    
    xi1, xi2 = dC.JackKnifeXi(rMax=rMax, fC=True)
    _, xi0_fit = np.abs(xi1[0].value), np.abs(xi2[0].value)
    
    return xi0_raw, xi0_fit, xi0_GPR

In [4]:
# Find a tile
expNum = DESutils.createBandDict("/data3/garyb/tno/y6/zone134")[1]["z"][0]
tilefiles = DESutils.findTiles(expNum, confirmTiles=True)
tab = tb.Table.read(tilefiles[0])

In [5]:
# Find match_ids for stars with mre than min_counts detections in the tile
min_counts = 50
match_ids, counts = np.unique(np.array(tab["MATCH_ID"]), return_counts=True)
match_ids = match_ids[counts > min_counts]

In [9]:
tol = 0.5*u.arcsec
for match_id in match_ids:
    star = tab[tab["MATCH_ID"] == match_id]
    
    SM = star["SPREAD_MODEL_I"][star["SPREAD_MODEL_I"] != 1e20].item()
    SEM = star["SPREADERR_MODEL_I"][star["SPREADERR_MODEL_I"] != 1e20].item()
    if np.abs(SM + (5/3)*SEM) >= 0.002:
        print(f"Likely not a star: {np.abs(SM + (5/3)*SEM)}")
        continue

    detections = []

    X = []
    Y = []
    RA0 = []
    DEC0 = []
    
    dX = []
    dY = []
    V = []
    raw002 = []
    fit002 = []
    GPR002 = []
    raw05 = []
    fit05 = []
    GPR05 = []
    
    for i, detection in enumerate(star):
        expNum = detection["EXPNUM"]
        band = detection["BAND"]

        # Check if this expNum has been analyzed
        GPRfile = glob.glob(f"/home/fortino/GPRSolutions/zone???/GPR.{expNum}.{band}RC.fits")
        if len(GPRfile) == 0:
            print(f"{expNum} {band} not analyzed.")
            continue
        elif len(GPRfile) > 1:
            print(f"Two files for {expNum} {band}... what???")
            continue
        else:
            print(f"Formatting {expNum} {band}...")
            dC = GPRutils.loadFITS(GPRfile[0])

        # Get ra0 and dec0 for gnomonic projection
        dC = GPRutils.loadFITS(GPRfile[0])
        E = tb.Table.read(dC.earthRef)
        E = E[E["expnum"] == dC.expNum]
        ra0 = E["ra"][0]
        dec0 = E["dec"][0]
        RA0.append(ra0)
        DEC0.append(dec0)
        
        X_GNOM, Y_GNOM = radec_to_gnomonic(
            np.array([detection["NEW_RA"]]), np.array([detection["NEW_DEC"]]),
            ra0, dec0)
        X.append(X_GNOM)
        Y.append(Y_GNOM)
        
        X_STAR = np.vstack([X_GNOM, Y_GNOM]).T
        
        # Initialize the TurbulentLayer object with the final parameters for this exposure.
        ttt = vk.TurbulentLayer(
            variance=dC.params[0],
            outerScale=dC.params[1],
            diameter=dC.params[2],
            wind=(dC.params[3], dC.params[4]))

        # Make K
        du, dv = GPRutils.getGrid(dC.Xtrain, dC.Xtrain)
        C = ttt.getCuv(du, dv)
        n1, n2 = C.shape[0], C.shape[1]
        K = np.swapaxes(C, 1, 2).reshape(2*n1, 2*n2)

        # Make the two White kernels.
        W_GAIA, W_DES = GPRutils.makeW(
            dC.Etrain_GAIA, dC.Etrain_DES,
            useRMS=dC.useRMS, curl=dC.curl)

        # Make Ks
        du, dv = GPRutils.getGrid(X_STAR, dC.Xtrain)
        Cs = ttt.getCuv(du, dv)
        n1, n2 = Cs.shape[0], Cs.shape[1]
        Ks = np.swapaxes(Cs, 1, 2).reshape(2*n1, 2*n2)

        # Find fbar_s
        cho_factor = scipy.linalg.cho_factor(K + W_GAIA + W_DES)
        alpha = scipy.linalg.cho_solve(cho_factor, GPRutils.flat(dC.Ytrain))
        fbar_s = GPRutils.unflat(np.dot(Ks.T, alpha))

        # Make Kss
        du, dv = GPRutils.getGrid(X_STAR, X_STAR)
        Css = ttt.getCuv(du, dv)
        n1, n2 = Css.shape[0], Css.shape[1]
        Kss = np.swapaxes(Css, 1, 2).reshape(2*n1, 2*n2)

        # Find V_s
        cho_factor = scipy.linalg.cho_factor(K + W_GAIA + W_DES)
        v = scipy.linalg.cho_solve(cho_factor, Ks)
        V_s = Kss - np.dot(v.T, v)

        xi0_002_raw, xi0_002_fit, xi0_002_GPR = getXiStuff(dC, 0.02*u.deg)
        xi0_05_raw, xi0_05_fit, xi0_05_GPR = getXiStuff(dC, 0.5*u.arcmin)
        
        dX.append(fbar_s[0, 0])
        dY.append(fbar_s[0, 1])
        V.append(V_s)

        raw002.append(xi0_002_raw)
        fit002.append(xi0_002_fit)
        GPR002.append(xi0_002_GPR)

        raw05.append(xi0_05_raw)
        fit05.append(xi0_05_fit)
        GPR05.append(xi0_05_GPR)
        
        detections.append(detection)

    tabf = tb.vstack(detections)
    RA0 = tb.Column(data=RA0, name="RA0")
    DEC0 = tb.Column(data=DEC0, name="DEC0")
    X = tb.Column(data=X, name="X_GNOM_NEW")
    Y = tb.Column(data=Y, name="Y_GNOM_NEW")
    
#     C_dXcosDec = tb.Column(
#         data=(dX/np.cos(eris_true["DEC"]*u.deg)).value, unit=u.mas,
#         name="dXcosDec",
#         description="First component of GP posterior predictive mean, divided by cos(Dec).")
    C_dX = tb.Column(
        data=dX, unit=u.mas,
        name="dX",
        description="First component of GP posterior predictive mean.")
    C_dY = tb.Column(
        data=dY, unit=u.mas,
        name="dY",
        description="Second component of GP posterior predictive mean.")
    C_V = tb.Column(
        data=V, unit=u.mas**2,
        name="V",
        description="2x2 posterior predictive covariance for the posterior predictive mean, (dX, dY).")

    C_raw002 = tb.Column(
        data=raw002, unit=u.mas**2,
        name="xi0_002_raw",
        description="All-pairs angle-averaged jackknifed xi+(r < 0.02 deg) for the raw data.")
    C_fit002 = tb.Column(
        data=fit002, unit=u.mas**2,
        name="xi0_002_fit",
        description="All-pairs angle-averaged jackknifed xi+(r < 0.02 deg) for the raw data with the fitted model removed.")
    C_GPR002 = tb.Column(
        data=GPR002, unit=u.mas**2,
        name="xi0_002_GPR",
        description="All-pairs angle-averaged jackknifed xi+(r < 0.02 deg) for the raw data with the final GPR model removed.")

    C_raw05 = tb.Column(
        data=raw05, unit=u.mas**2,
        name="xi0_05_raw",
        description="All-pairs angle-averaged jackknifed xi+(r < 0.5 arcmin) for the raw data.")
    C_fit05 = tb.Column(
        data=fit05, unit=u.mas**2,
        name="xi0_05_fit",
        description="All-pairs angle-averaged jackknifed xi+(r < 0.5 arcmin) for the raw data with the fitted model removed.")
    C_GPR05 = tb.Column(
        data=GPR05, unit=u.mas**2,
        name="xi0_05_GPR",
        description="All-pairs angle-averaged jackknifed xi+(r < 0.5 arcmin) for the raw data with the final GPR model removed.")
    
    cols = [RA0, DEC0, X, Y, C_dX, C_dY, C_V, C_raw002, C_fit002, C_GPR002, C_raw05, C_fit05, C_GPR05]
    tabf.add_columns(cols)
    break

Formatting 689611 i...
Formatting 579816 g...
Formatting 355302 Y...
999999 0.0 not analyzed.
370603 r not analyzed.
Formatting 573796 Y...
362368 i not analyzed.
367487 g not analyzed.
Formatting 579815 r...
Formatting 689613 z...
Formatting 362365 i...
Formatting 348819 z...
Formatting 348819 z...
Formatting 573398 z...
Formatting 244218 Y...
Formatting 371369 g...
370603 r not analyzed.
Formatting 364210 i...
Formatting 576862 i...
Formatting 248717 g...
Formatting 367488 r...
Formatting 372522 g...
Formatting 689612 r...
Formatting 689612 r...
Formatting 372522 g...
Formatting 479272 Y...
362368 i not analyzed.
Formatting 362366 r...
367487 g not analyzed.
Formatting 370601 g...
Formatting 364209 z...
Formatting 370601 g...
Formatting 364209 z...
Formatting 371369 g...
Formatting 370200 r...
Formatting 370200 r...
Formatting 573398 z...
Formatting 372006 z...
Formatting 372064 g...
Formatting 592152 z...
Formatting 372064 g...
Formatting 576861 r...
Formatting 370609 i...
Formattin

In [11]:
tabf.info()

<Table length=61>
       name        dtype  shape  unit                                                description                                                
----------------- ------- ------ ---- ----------------------------------------------------------------------------------------------------------
               RA float64                                                                                                                       
              DEC float64                                                                                                                       
    ERRAWIN_WORLD float64                                                                                                                       
       XWIN_IMAGE float64                                                                                                                       
       YWIN_IMAGE float64                                                                                       

In [18]:
tabf.write(f"/home/fortino/DESworkspace/data/{'GPR_'+os.path.basename(tilefiles[0])}", format="fits")